# Access ENVI formatted file from NASA Earthdata Cloud

## Setup environment

Follow instruction here to setup NASA Earthdata access: https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget

Install rasterio 
```bash
pip install rasterio[s3]
```

## AVIRIS-NG File
S3 links to AVIRIS-NG file can be obtained from NASA Earthdata search: https://search.earthdata.nasa.gov/search/granules?p=C2659129205-ORNL_CLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=AVIRIS-NG&fpj=AVIRIS&tl=1689178477.751!3!!

In [2]:
# import modules
import boto3
import requests
import rasterio
import rioxarray
from os import path

# avirisng 
s3_url = 's3://ornl-cumulus-prod-protected/aviris/AVIRIS-NG_L2_Reflectance/data/ang20140621t190550_h2o_v1g_img.bin'

# s3 credentials
earthadata_s3 = f"https://data.ornldaac.earthdata.nasa.gov/s3credentials"
r = requests.get(earthadata_s3)
r.raise_for_status()
s3credentials = r.json()

# aws session using rasterio
session = boto3.Session(aws_access_key_id=s3credentials['accessKeyId'], 
                        aws_secret_access_key=s3credentials['secretAccessKey'],
                        aws_session_token=s3credentials['sessionToken'],
                        region_name='us-west-2')

env = rasterio.Env(rasterio.session.AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='TRUE',
                  GDAL_HTTP_COOKIEFILE=path.expanduser('~/.urs_cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=path.expanduser('~/.urs_cookies.txt'))
env.__enter__()
avirisng = rioxarray.open_rasterio(s3_url, driver='ENVI', chuncks=True, parse_coordinates=True)
avirisng

<xarray.DataArray (band: 3, y: 2413, x: 736)>
[5327904 values with dtype=float32]
Coordinates:
  * band         (band) int64 1 2 3
    xc           (y, x) float64 6.299e+05 6.299e+05 ... 6.37e+05 6.37e+05
    yc           (y, x) float64 3.928e+06 3.928e+06 ... 3.922e+06 3.922e+06
    spatial_ref  int64 0
Dimensions without coordinates: y, x
Attributes: (12/22)
    Band_1:             {Column water vapor (346.2995778 Nanometers)
    Band_2:             Liquid absorption path (351.3082478 Nanometers)
    Band_3:             Ice absorption path (356.3169178 Nanometers)
    wavelength_units:   Nanometers
    wavelength:         [ 346.2995778  351.3082478  356.3169178  361.3255878 ...
    _FillValue:         -9999.0
    ...                 ...
    fwhm:               [5.48425 5.48856 5.49283 5.49705 5.50122 5.50536 5.50...
    header_offset:      0
    interleave:         bil
    lines:              2413
    samples:            736
    smoothing_factors:   1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 ...

In [3]:
# exit session
env.__exit__()